In [32]:
import torch
import json
import pandas as pd
with open('../../meta/whisper_special_tokens.json') as f:
    SPECIAL_TOKENS = json.load(f)
LANG_TOKENS = SPECIAL_TOKENS['lang']
LANG_TOKEN_IDS = [lang_obj['id'] for lang_obj in LANG_TOKENS.values()]
MIN_LANG_ID = min(LANG_TOKEN_IDS)

In [33]:
lid_df = pd.DataFrame(
    {
        'id': lang_obj['id'],
        'rel_id': lang_obj['id']-MIN_LANG_ID,
        'lang': lang,
    } for lang, lang_obj in LANG_TOKENS.items()
)
lid_df

,id,rel_id,lang
0,50259,0,en
1,50260,1,zh
2,50261,2,de
3,50262,3,es
4,50263,4,ru
...,...,...,...
94,50353,94,ln
95,50354,95,ha
96,50355,96,ba
97,50356,97,jw


In [4]:
lid_logits_path = '../../data/tira-clean-split_lid_logits.pt'
lid_logits = torch.load(lid_logits_path)
len(lid_logits)

99

In [7]:
lid_logits['en'].shape

torch.Size([16384])

In [19]:
logit_mat = torch.zeros(16384, 99)
logit_mat

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [23]:
for lang, logit_tensor in lid_logits.items():
    lang_id=LANG_TOKENS[lang]['id']-MIN_LANG_ID
    logit_mat[:,lang_id]=logit_tensor
prob_mat = logit_mat.softmax(dim=1)
prob_mat[0].sum(), prob_mat.shape

(tensor(1.0000), torch.Size([16384, 99]))

In [34]:
lid_df['prob_mean']=prob_mat.mean(dim=0)
lid_df['prob_std']=prob_mat.std(dim=0)

In [40]:
lid_df.sort_values('prob_mean', ascending=False)

,id,rel_id,lang,prob_mean,prob_std
0,50259,0,en,0.079624,0.090021
3,50262,3,es,0.075314,0.143020
59,50318,59,sw,0.057751,0.140158
63,50322,63,si,0.051566,0.106907
13,50272,13,ar,0.048845,0.119238
...,...,...,...,...,...
86,50345,86,lb,0.000058,0.000226
90,50349,90,mg,0.000049,0.000090
82,50341,82,tk,0.000010,0.000016
78,50337,78,uz,0.000009,0.000034
